In [3]:
import pandas as pd
import json

In [40]:
#filenameIn = "data/goldpaths/goldsequences-0.json"
filenameIn = "data/goldpaths/goldsequences-0-1-2-3-4-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29.json"
f = open(filenameIn)
data = json.load(f)
print("Tasks stored in this gold data: " + str(data.keys()))
numMoves = 0
numSequences = 0
all_observation = []


Tasks stored in this gold data: dict_keys(['0', '1', '2', '3', '4', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29'])


In [66]:
def get_observation_look(text_observation: str, inventory: bool = False) -> list:
    ignore_terms = ['You also see:', '(', ')', 'In it, you see:', 'In your inventory, you see:']
    text = text_observation
    for ignore in ignore_terms:
        text = text.replace(ignore, '')
    all_beliefs = [sentence.replace('\t', '') for sentence in text.split('\n') if len(sentence) > 0]
    if inventory:
        return [f"You have {s.lower()} in your inventory" for s in all_beliefs]
    else:
        return all_beliefs

In [67]:
agent_information = []
for taskIdx in data.keys():
    taskData = data[taskIdx]
    taskName = taskData['taskName']
    goldSequences = taskData['goldActionSequences']
    for goldSequence in goldSequences:
        fold = goldSequence['fold']
        path = goldSequence['path']
        for step in path:
            action = step['action']
            observation = step['observation']
            freelook = step['freelook']
            inventory = step['inventory']
            all_obs = get_observation_look(observation)
            all_inventory_obs = get_observation_look(inventory, inventory=True)
            agent_information += all_obs
            agent_information += all_inventory_obs

data_df = pd.DataFrame(agent_information)
data_df.head()

,0
0,This room is called the hallway.
1,a picture
2,a substance called air
3,the agent
4,A door to the green house that is open


In [69]:
len(agent_information), len(list(set(agent_information)))

(1623023, 23108)

In [72]:
data_sentence = []
for sentence in list(set(agent_information)):
    data_sentence.append({
        "sentence": sentence,
        "num_words": len(sentence.split())
    })
sentence_df = pd.DataFrame(data_sentence)
sentence_df.head()

,sentence,num_words
0,"a flower pot 9 containing soil, a pea plant in...",32
1,an inclined plane with a unknown material G su...,18
2,the thermometer measures a temperature of 112 ...,9
3,"a flower pot 5 containing soil, a unknown D pl...",28
4,"a table. On the table is: a switch, which is o...",43


In [74]:
sentence_df.sort_values("num_words", ascending=False)

,sentence,num_words
21705,"a flower pot 1 containing soil, a unknown E pl...",78
8352,a flower pot 4 containing a unknown E plant in...,77
3244,"a flower pot 9 containing soil, a unknown E pl...",76
19437,You have a seed jar containing a unknown e pla...,76
1150,"a flower pot 1 containing soil, a unknown E pl...",75
...,...,...
12755,chocolate,1
5595,marshmallow,1
9695,bread,1
17824,flour,1


In [75]:
sentence_df.to_csv("data/dataset_sentence_level.csv", index=False)